In [3]:
import os
import sys

# caution: path[0] is reserved for script path (or '' in REPL).
sys.path.insert(1, os.path.abspath("./../src"))


import datetime
import importlib

import astropy.time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tqdm
from cdflib.epochs_astropy import CDFAstropy as cdfepoch

import data_loader
import rbsp_chorus_tool

importlib.reload(data_loader)
importlib.reload(rbsp_chorus_tool)

pdata_folder = os.path.abspath(r"./../processed_data/chorus_neural_network/")

%matplotlib qt

In [10]:
# STAGE 0 DATA VERIFICATION FOR POES LSTAR CALCULATIONS

mpe_folder = os.path.join(pdata_folder, "STAGE_0", "MPE_DATA_PREPROCESSED_WITH_LSTAR")

year = 2012
SATID = "m02"

refs = np.load(
    file=os.path.join(
        mpe_folder,
        rf"MPE_PREPROCESSED_DATA_T89_{year}.npz",
    ),
    allow_pickle=True,
)

DATA = refs["DATA"].flatten()[0]
SAT = DATA[SATID]

dt_for_all = np.array([datetime.datetime.fromtimestamp(t) for t in SAT["UNIX_TIME"]])

plt.plot(dt_for_all, SAT["Lstar"], label="L*", color="red", marker="*")
plt.plot(dt_for_all, SAT["L"], label="IGRF Lm", color="black", marker="*")
plt.ylabel("|L|")
plt.xlabel("Time")
plt.title(f"Some Orbits for {SATID} in {year}")
plt.legend()

plt.show()

C:\Dev\Research\Research_Tools\ResearchPy\Lib\site-packages\ipykernel\eventloops.py:145: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  el.exec() if hasattr(el, "exec") else el.exec_()


In [2]:
# DATA VERIFICATION RBSP L-STAR DATA CALCULATIONS

lstar_folder = os.path.join(pdata_folder, "STAGE_1", "Lstar")

year = 2013
sat = "a"
refs = np.load(
    file=os.path.join(lstar_folder, rf"RBSP_{sat.upper()}_T89_{year}.npz"),
    allow_pickle=True,
)

OMNI = data_loader.load_raw_data_from_config(
    id=["OMNI", "ONE_HOUR_RESOLUTION"],
    start=datetime.datetime(year=year, month=1, day=1),
    end=datetime.datetime(year=year + 1, month=1, day=1),
)

OMNI_TIME = cdfepoch.unixtime(OMNI["Epoch"])
KP = OMNI["KP"].astype(np.float64)

invalid_omni_times = (
    (OMNI_TIME < 0) | (KP < 0) | (KP >= 99) | np.isnan(KP) | np.isnan(OMNI_TIME)
)
KP[invalid_omni_times] = np.nan


KP_INTERPOLATED = np.interp(refs["UNIX_TIME"], OMNI_TIME, KP, left=np.nan, right=np.nan)


fig, axs = plt.subplots(2, 1, sharex=True)

dates = np.array([datetime.datetime.fromtimestamp(t) for t in refs["UNIX_TIME"]])


axs[0].plot(dates, refs["Lstar"])
axs[1].plot(dates, KP_INTERPOLATED)
plt.xlabel("Time (UTC)")
axs[0].set_ylabel("L*")
axs[1].set_ylabel("KP-Index")

plt.show()

C:\Dev\Research\Research_Tools\ResearchPy\Lib\site-packages\erfa\core.py:133: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  warn(f'ERFA function "{func_name}" yielded {wmsg}', ErfaWarning)


In [ ]:
# Interpolate POES data cause I forgot to interpolate it before with the nans included

for _year in range(2012, 2021):

    POES = {}

    refs = np.load(
        rf"./../processed_data/chorus_neural_network/STAGE_0/MPE_DATA_PREPROCESSED_WITH_LSTAR/MPE_PREPROCESSED_DATA_T89_{_year}.npz",
        allow_pickle=True,
    )
    POES_DATA = refs["DATA"].flatten()[0]

    for SATID in POES_DATA:

        SAT = POES_DATA[SATID]

        UNIX_TIME = []
        LSTAR = []
        MLT = []
        BLC_FLUX_0 = []
        BLC_FLUX_1 = []
        BLC_FLUX_2 = []
        BLC_FLUX_3 = []
        BLC_FLUX_4 = []
        BLC_FLUX_5 = []
        BLC_FLUX_6 = []
        BLC_FLUX_7 = []

        for p in range(len(SAT["UNIX_TIME"]) - 1):

            t1 = SAT["UNIX_TIME"][p]
            t2 = SAT["UNIX_TIME"][p + 1]

            if t2 - t1 < 30.0:

                t_points = np.arange(t1, t2 + 1, step=1, dtype=np.float64)

                UNIX_TIME.append(t_points)
                LSTAR.append(
                    np.interp(
                        x=t_points,
                        xp=[t1, t2],
                        fp=[SAT["Lstar"][p], SAT["Lstar"][p + 1]],
                        left=np.nan,
                        right=np.nan,
                    )
                )

                X_INTERPOLATED = np.interp(
                    t_points,
                    xp=[t1, t2],
                    fp=[
                        np.cos(SAT["MLT"][p] * 2 * np.pi / 24.0),
                        np.cos(SAT["MLT"][p + 1] * 2 * np.pi / 24.0),
                    ],
                    left=np.nan,
                    right=np.nan,
                )
                Y_INTERPOLATED = np.interp(
                    t_points,
                    xp=[t1, t2],
                    fp=[
                        np.sin(SAT["MLT"][p] * 2 * np.pi / 24.0),
                        np.sin(SAT["MLT"][p + 1] * 2 * np.pi / 24.0),
                    ],
                    left=np.nan,
                    right=np.nan,
                )
                ANGLE_IN_RADIANS = np.mod(
                    np.arctan2(Y_INTERPOLATED, X_INTERPOLATED) + 2 * np.pi, 2 * np.pi
                )

                MLT.append((ANGLE_IN_RADIANS * 24.0) / (2 * np.pi))

                BLC_FLUX_0.append(
                    np.interp(
                        x=t_points,
                        xp=[t1, t2],
                        fp=[SAT["BLC_Flux"][p, 0], SAT["BLC_Flux"][p + 1, 0]],
                        left=np.nan,
                        right=np.nan,
                    )
                )
                BLC_FLUX_1.append(
                    np.interp(
                        x=t_points,
                        xp=[t1, t2],
                        fp=[SAT["BLC_Flux"][p, 1], SAT["BLC_Flux"][p + 1, 1]],
                        left=np.nan,
                        right=np.nan,
                    )
                )
                BLC_FLUX_2.append(
                    np.interp(
                        x=t_points,
                        xp=[t1, t2],
                        fp=[SAT["BLC_Flux"][p, 2], SAT["BLC_Flux"][p + 1, 2]],
                        left=np.nan,
                        right=np.nan,
                    )
                )
                BLC_FLUX_3.append(
                    np.interp(
                        x=t_points,
                        xp=[t1, t2],
                        fp=[SAT["BLC_Flux"][p, 3], SAT["BLC_Flux"][p + 1, 3]],
                        left=np.nan,
                        right=np.nan,
                    )
                )
                BLC_FLUX_4.append(
                    np.interp(
                        x=t_points,
                        xp=[t1, t2],
                        fp=[SAT["BLC_Flux"][p, 4], SAT["BLC_Flux"][p + 1, 4]],
                        left=np.nan,
                        right=np.nan,
                    )
                )
                BLC_FLUX_5.append(
                    np.interp(
                        x=t_points,
                        xp=[t1, t2],
                        fp=[SAT["BLC_Flux"][p, 5], SAT["BLC_Flux"][p + 1, 5]],
                        left=np.nan,
                        right=np.nan,
                    )
                )
                BLC_FLUX_6.append(
                    np.interp(
                        x=t_points,
                        xp=[t1, t2],
                        fp=[SAT["BLC_Flux"][p, 6], SAT["BLC_Flux"][p + 1, 6]],
                        left=np.nan,
                        right=np.nan,
                    )
                )
                BLC_FLUX_7.append(
                    np.interp(
                        x=t_points,
                        xp=[t1, t2],
                        fp=[SAT["BLC_Flux"][p, 7], SAT["BLC_Flux"][p + 1, 7]],
                        left=np.nan,
                        right=np.nan,
                    )
                )

        UNIX_TIME = np.hstack(UNIX_TIME)
        LSTAR = np.hstack(LSTAR)
        MLT = np.hstack(MLT)
        BLC_FLUX_0 = np.hstack(BLC_FLUX_0)
        BLC_FLUX_1 = np.hstack(BLC_FLUX_1)
        BLC_FLUX_2 = np.hstack(BLC_FLUX_2)
        BLC_FLUX_3 = np.hstack(BLC_FLUX_3)
        BLC_FLUX_4 = np.hstack(BLC_FLUX_4)
        BLC_FLUX_5 = np.hstack(BLC_FLUX_5)
        BLC_FLUX_6 = np.hstack(BLC_FLUX_6)
        BLC_FLUX_7 = np.hstack(BLC_FLUX_7)
        BLC_FLUX = np.hstack(
            [
                np.expand_dims(BLC_FLUX_0, axis=1),
                np.expand_dims(BLC_FLUX_1, axis=1),
                np.expand_dims(BLC_FLUX_2, axis=1),
                np.expand_dims(BLC_FLUX_3, axis=1),
                np.expand_dims(BLC_FLUX_4, axis=1),
                np.expand_dims(BLC_FLUX_5, axis=1),
                np.expand_dims(BLC_FLUX_6, axis=1),
                np.expand_dims(BLC_FLUX_7, axis=1),
            ]
        )

        POES[SATID] = {
            "UNIX_TIME": UNIX_TIME,
            "MLT": MLT,
            "BLC_Flux": BLC_FLUX,
            "LSTAR": LSTAR,
        }

    if not POES:
        print(f"No POES satellite coverage found for year : {_year}")
        print(f"SKIPPING YEAR : {_year}")
        continue

    refs.close()

    np.savez(
        file=os.path.abspath(
            f"./../processed_data/chorus_neural_network/STAGE_0/MPE_DATA_PREPROCESSED_WITH_LSTAR/MPE_PREPROCESSED_DATA_T89_{_year}_interpolated.npz"
        ),
        DATA=POES,
    )

In [2]:
%%time
# Stage 1 RBSP Chorus Preprocessing, Obtains clean chorus amplitudes

year = 2019

start = datetime.datetime(year=year, month=1, day=1)
end = datetime.datetime(year=year, month=10, day=13, hour=23, minute=59, second=59)

evenly_spaced_seconds = np.arange(start.timestamp(), end.timestamp() + 1, step=1)


WNA_a = data_loader.load_raw_data_from_config(
    id=["RBSP", "EMFISIS", "L4", "WNA_SURVEY"],
    start=start,
    end=end,
    satellite="a",
    root_data_dir="/project/rbsp/data/",
    use_config_keys_in_subdir=False,
)

WNA_b = data_loader.load_raw_data_from_config(
    id=["RBSP", "EMFISIS", "L4", "WNA_SURVEY"],
    start=start,
    end=end,
    satellite="b",
    root_data_dir="/project/rbsp/data/",
    use_config_keys_in_subdir=False,
)

WFR_a = data_loader.load_raw_data_from_config(
    id=["RBSP", "EMFISIS", "L2", "WFR_SPECTRAL_MATRIX_DIAGONAL"],
    start=start,
    end=end,
    satellite="a",
    root_data_dir="/project/rbsp/data/",
    use_config_keys_in_subdir=False,
)

WFR_b = data_loader.load_raw_data_from_config(
    id=["RBSP", "EMFISIS", "L2", "WFR_SPECTRAL_MATRIX_DIAGONAL"],
    start=start,
    end=end,
    satellite="b",
    root_data_dir="/project/rbsp/data/",
    use_config_keys_in_subdir=False,
)

num_wna_files_A = len(WNA_a["timestamps_per_file"])
num_wna_files_B = len(WNA_b["timestamps_per_file"])
num_wfr_files_A = WFR_a["WFR_bandwidth"].shape[0]
num_wfr_files_B = WFR_b["WFR_bandwidth"].shape[0]

print(
    f"Number of files loaded: {num_wna_files_A, num_wna_files_B, num_wfr_files_A, num_wfr_files_B}"
)

if len({num_wna_files_A, num_wfr_files_A}) != 1:
    raise Exception("The same number of days wasn't loaded for RBSP-A!")

if len({num_wna_files_B, num_wfr_files_B}) != 1:
    raise Exception("The same number of days wasn't loaded for RBSP-B!")

Number of files loaded: (286, 197, 286, 197)
CPU times: user 9.66 s, sys: 13.4 s, total: 23 s
Wall time: 2min 20s


In [3]:
%%time
MLT_A = WNA_a["MLT"]
L_A = WNA_a["L"]
EPOCH_A = WNA_a["Epoch"]

TIME_A = astropy.time.Time(cdfepoch.to_datetime(EPOCH_A), format="datetime").unix

CHORUS_A = rbsp_chorus_tool.iterate_through_days_and_calculate_chorus_amplitudes(
    WNA_survey=WNA_a, WFR_spectral_matrix=WFR_a
)

LOWER_CHORUS_A = CHORUS_A["Lower_Band"]
UPPER_CHORUS_A = CHORUS_A["Upper_Band"]

within_epoch_range_A = (start.timestamp() < TIME_A) & (TIME_A < end.timestamp())
finite_chorus_A = np.isfinite(LOWER_CHORUS_A) & np.isfinite(UPPER_CHORUS_A)
all_valid_coordinates_A = (
    (EPOCH_A > 0) & (0 <= MLT_A) & (MLT_A <= 24) & (0 < L_A) & (L_A < 10)
)

L_A[~(within_epoch_range_A & finite_chorus_A & all_valid_coordinates_A)] = np.nan
MLT_A[~(within_epoch_range_A & finite_chorus_A & all_valid_coordinates_A)] = np.nan
LOWER_CHORUS_A[~(within_epoch_range_A & finite_chorus_A & all_valid_coordinates_A)] = (
    np.nan
)
UPPER_CHORUS_A[~(within_epoch_range_A & finite_chorus_A & all_valid_coordinates_A)] = (
    np.nan
)

L_A = np.interp(x=evenly_spaced_seconds, xp=TIME_A, fp=L_A)
x_int_A = np.interp(
    evenly_spaced_seconds,
    xp=TIME_A,
    fp=np.cos(MLT_A * 2 * np.pi / 24.0),
    left=np.nan,
    right=np.nan,
)
y_int_A = np.interp(
    evenly_spaced_seconds,
    xp=TIME_A,
    fp=np.sin(MLT_A * 2 * np.pi / 24.0),
    left=np.nan,
    right=np.nan,
)
angle_A = np.mod(np.arctan2(y_int_A, x_int_A) + 2 * np.pi, 2 * np.pi)
MLT_A = (angle_A * 24) / (2 * np.pi)

LOWER_CHORUS_A = np.interp(x=evenly_spaced_seconds, xp=TIME_A, fp=LOWER_CHORUS_A)
UPPER_CHORUS_A = np.interp(x=evenly_spaced_seconds, xp=TIME_A, fp=UPPER_CHORUS_A)

NOT_NAN_A = (
    np.isfinite(L_A)
    & np.isfinite(MLT_A)
    & np.isfinite(LOWER_CHORUS_A)
    & np.isfinite(UPPER_CHORUS_A)
)

L_A = L_A[NOT_NAN_A]
MLT_A = MLT_A[NOT_NAN_A]
LOWER_CHORUS_A = LOWER_CHORUS_A[NOT_NAN_A]
UPPER_CHORUS_A = UPPER_CHORUS_A[NOT_NAN_A]

TIME_A = evenly_spaced_seconds[NOT_NAN_A]

# -----------------------------------------------------------------------------------------------------------------------------

MLT_B = WNA_b["MLT"]
L_B = WNA_b["L"]
EPOCH_B = WNA_b["Epoch"]
TIME_B = astropy.time.Time(cdfepoch.to_datetime(EPOCH_B), format="datetime").unix

CHORUS_B = rbsp_chorus_tool.iterate_through_days_and_calculate_chorus_amplitudes(
    WNA_survey=WNA_b, WFR_spectral_matrix=WFR_b
)

LOWER_CHORUS_B = CHORUS_B["Lower_Band"]
UPPER_CHORUS_B = CHORUS_B["Upper_Band"]

within_epoch_range_B = (start.timestamp() < TIME_B) & (TIME_B < end.timestamp())
finite_chorus_B = np.isfinite(LOWER_CHORUS_B) & np.isfinite(UPPER_CHORUS_B)
all_valid_coordinates_B = (
    (EPOCH_B > 0) & (0 <= MLT_B) & (MLT_B <= 24) & (0 < L_B) & (L_B < 10)
)

L_B[~(within_epoch_range_B & finite_chorus_B & all_valid_coordinates_B)] = np.nan
MLT_B[~(within_epoch_range_B & finite_chorus_B & all_valid_coordinates_B)] = np.nan
LOWER_CHORUS_B[~(within_epoch_range_B & finite_chorus_B & all_valid_coordinates_B)] = (
    np.nan
)
UPPER_CHORUS_B[~(within_epoch_range_B & finite_chorus_B & all_valid_coordinates_B)] = (
    np.nan
)

L_B = np.interp(x=evenly_spaced_seconds, xp=TIME_B, fp=L_B)

X_INTERPOLATED_B = np.interp(
    evenly_spaced_seconds,
    xp=TIME_B,
    fp=np.cos(MLT_B * 2 * np.pi / 24.0),
    left=np.nan,
    right=np.nan,
)
Y_INTERPOLATED_B = np.interp(
    evenly_spaced_seconds,
    xp=TIME_B,
    fp=np.sin(MLT_B * 2 * np.pi / 24.0),
    left=np.nan,
    right=np.nan,
)
ANGLE_IN_RADIANS_B = np.mod(
    np.arctan2(Y_INTERPOLATED_B, X_INTERPOLATED_B) + 2 * np.pi, 2 * np.pi
)
MLT_B = (ANGLE_IN_RADIANS_B * 24) / (2 * np.pi)

LOWER_CHORUS_B = np.interp(x=evenly_spaced_seconds, xp=TIME_B, fp=LOWER_CHORUS_B)
UPPER_CHORUS_B = np.interp(x=evenly_spaced_seconds, xp=TIME_B, fp=UPPER_CHORUS_B)

NOT_NAN_B = (
    np.isfinite(L_B)
    & np.isfinite(MLT_B)
    & np.isfinite(LOWER_CHORUS_B)
    & np.isfinite(UPPER_CHORUS_B)
)

L_B = L_B[NOT_NAN_B]
MLT_B = MLT_B[NOT_NAN_B]
LOWER_CHORUS_B = LOWER_CHORUS_B[NOT_NAN_B]
UPPER_CHORUS_B = UPPER_CHORUS_B[NOT_NAN_B]
TIME_B = evenly_spaced_seconds[NOT_NAN_B]
# -----------------------------------------------------------------------------------------------------------------------------

print("Shapes for A:")
print(LOWER_CHORUS_A.shape)
print(UPPER_CHORUS_A.shape)
print(L_A.shape)
print(MLT_A.shape)

print("\n")

print("Shapes for B:")
print(LOWER_CHORUS_B.shape)
print(UPPER_CHORUS_B.shape)
print(L_B.shape)
print(MLT_B.shape)

Shapes for A:
(17660469,)
(17660469,)
(17660469,)
(17660469,)


Shapes for B:
(12247254,)
(12247254,)
(12247254,)
(12247254,)
CPU times: user 2min 48s, sys: 5.39 s, total: 2min 54s
Wall time: 2min 52s


In [4]:
# Save the RBSP stage 1 data, might honestly only need one stage

np.savez(
    file=os.path.abspath(
        f"./../processed_data/chorus_neural_network/STAGE_1/CHORUS/RBSP_OBSERVED_CHORUS_{year}.npz"
    ),
    UNIX_TIME_A=TIME_A,
    MLT_A=MLT_A,
    L_A=L_A,
    LOWER_BAND_CHORUS_A=LOWER_CHORUS_A,
    UPPER_BAND_CHORUS_A=UPPER_CHORUS_A,
    UNIX_TIME_B=TIME_B,
    MLT_B=MLT_B,
    L_B=L_B,
    LOWER_BAND_CHORUS_B=LOWER_CHORUS_B,
    UPPER_BAND_CHORUS_B=UPPER_CHORUS_B,
)

In [ ]:
# Stage 2, clean then combine RBSP, OMNI, and POES Data and find conjunctions between RBSP and POES See find_conjunctions_rbsp_poes.py

In [9]:
# Stage 3, Look at the data and make sure its good enough, then remove solar proton events
VERSION = "v1c"
FIELD_MODEL = "T89"

pdata_folder = os.path.abspath(r"./../processed_data/chorus_neural_network/")

STAGE_2_folder = os.path.join(pdata_folder, "STAGE_2", VERSION)

CONJUNCTIONS_REFS = np.load(
    file=os.path.join(STAGE_2_folder, rf"CONJUNCTIONS_{VERSION}_{FIELD_MODEL}.npz")
)

CONJUNCTIONS_TESTING = CONJUNCTIONS_REFS["CONJUNCTIONS"]

CONJUNCTIONS_REFS.close()

In [10]:
"""CONJUNCTION =  [UNIX_TIME,
LSTAR,
MLT,
*FLUX_SPECTRUM,
TOTAL_TIME / NUM_CANDIDATES, #TIME OF RBSP POINT CHOSEN
TOTAL_LSTAR / NUM_CANDIDATES, #LSTAR OF RBSP POINT CHOSEN
TOTAL_DEL_MLT / NUM_CANDIDATES, #DIFFERENCE IN MLT FOUND
TOTAL_UPPER_BAND / NUM_CANDIDATES, #UPPER BAND CHORUS OBSERVED
TOTAL_LOWER_BAND / NUM_CANDIDATES, #LOWER BAND CHORUS OBSERVED
AVG_SME,
AVG_AVG_B,
AVG_FLOW_SPEED,
AVG_PROTON_DENSITY,
AVG_SYM_H]"""

C_POES_TIME = CONJUNCTIONS_TESTING[:, 0]
C_POES_LSTAR = CONJUNCTIONS_TESTING[:, 1]
C_POES_MLT = CONJUNCTIONS_TESTING[:, 2]
C_POES_FLUX = CONJUNCTIONS_TESTING[:, 3:-10]
C_RBSP_TIME = CONJUNCTIONS_TESTING[:, -10]
C_RBSP_LSTAR = CONJUNCTIONS_TESTING[:, -9]
C_RBSP_DEL_MLT = CONJUNCTIONS_TESTING[:, -8]
C_RBSP_UPPER_BAND = CONJUNCTIONS_TESTING[:, -7]
C_RBSP_LOWER_BAND = CONJUNCTIONS_TESTING[:, -6]
C_AVG_SME = CONJUNCTIONS_TESTING[:, -5]
C_AVG_AVG_B = CONJUNCTIONS_TESTING[:, -4]
C_AVG_FLOW_SPEED = CONJUNCTIONS_TESTING[:, -3]
C_AVG_PROTON_DENSITY = CONJUNCTIONS_TESTING[:, -2]
C_AVG_SYM_H = CONJUNCTIONS_TESTING[:, -1]

with open(os.path.join(STAGE_2_folder, rf"CONJUNCTIONS_{VERSION}_{FIELD_MODEL}.txt"), "w",) as f:

    f.write("\nConjunctions:\n")
    f.write(f"Number of conjunctions: {CONJUNCTIONS_TESTING.shape[0]} [#]\n")
    f.write(f"Minimum RBSP Time: {np.min(C_RBSP_TIME)} [seconds since unix epoch]\n")
    f.write(f"Maximum RBSP Time: {np.max(C_RBSP_TIME)} [seconds since unix epoch]\n")
    f.write(f"Minimum POES Time: {np.min(C_POES_TIME)} [seconds since unix epoch]\n")
    f.write(f"Maximum POES Time: {np.max(C_POES_TIME)} [seconds since unix epoch]\n")

    f.write("\nL:\n")
    f.write(f"Mean Difference: {np.mean(C_POES_LSTAR - C_RBSP_LSTAR)} [L]\n")
    f.write(
        f"Standard deviation of Difference {np.std(C_POES_LSTAR - C_RBSP_LSTAR)} [L]\n"
    )
    f.write(
        f"Minimum Absolute Difference : {np.min(np.abs(C_POES_LSTAR - C_RBSP_LSTAR))} [L]\n"
    )
    f.write(
        f"Maximum Absolute Difference : {np.max(np.abs(C_POES_LSTAR - C_RBSP_LSTAR))} [L]\n"
    )

    f.write("\nMLT: \n")
    f.write(f"Mean Absolute Difference: {np.mean(C_RBSP_DEL_MLT)} [MLT]\n")
    f.write(
        f"Standard deviation of Absolute Difference {np.std(C_RBSP_DEL_MLT)} [MLT]\n"
    )
    f.write(f"Minimum Absolute Difference : {np.min(np.abs(C_RBSP_DEL_MLT))} [MLT]\n")
    f.write(f"Maximum Absolute Difference : {np.max(np.abs(C_RBSP_DEL_MLT))} [MLT]\n")

    f.write("\nTime: \n")
    f.write(f"Mean Difference: {np.mean(C_POES_TIME - C_RBSP_TIME)} [s]\n")
    f.write(
        f"Standard deviation of Difference {np.std(C_POES_TIME - C_RBSP_TIME)} [s]\n"
    )
    f.write(
        f"Minimum Absolute Difference : {np.min(np.abs(C_POES_TIME - C_RBSP_TIME))} [s]\n"
    )
    f.write(
        f"Maximum Absolute Difference : {np.max(np.abs(C_POES_TIME - C_RBSP_TIME))} [s]\n"
    )

    f.write("\nUpper Band Chorus: \n")
    f.write(f"Mean: {np.mean(C_RBSP_UPPER_BAND)} [pT]\n")
    f.write(f"Standard Deviation: {np.std(C_RBSP_UPPER_BAND)} [pT]\n")
    f.write(f"Minimum: {np.min(C_RBSP_UPPER_BAND)} [pT]\n")
    f.write(f"Maximum: {np.max(C_RBSP_UPPER_BAND)} [pT]\n")

    f.write("\nLower Band Chorus: \n")
    f.write(f"Mean: {np.mean(C_RBSP_LOWER_BAND)} [pT]\n")
    f.write(f"Standard Deviation: {np.std(C_RBSP_LOWER_BAND)} [pT]\n")
    f.write(f"Minimum: {np.min(C_RBSP_LOWER_BAND)} [pT]\n")
    f.write(f"Maximum: {np.max(C_RBSP_LOWER_BAND)} [pT]\n")

    f.write("\nSME: \n")
    f.write(f"Mean: {np.mean(C_AVG_SME)} [nT]\n")
    f.write(f"Standard Deviation: {np.std(C_AVG_SME)} [nT]\n")
    f.write(f"Minimum: {np.min(C_AVG_SME)} [nT]\n")
    f.write(f"Maximum: {np.max(C_AVG_SME)} [nT]\n")

    f.write("\nAVG_B: \n")
    f.write(f"Mean: {np.mean(C_AVG_AVG_B)} [nT]\n")
    f.write(f"Standard Deviation: {np.std(C_AVG_AVG_B)} [nT]\n")
    f.write(f"Minimum: {np.min(C_AVG_AVG_B)} [nT]\n")
    f.write(f"Maximum: {np.max(C_AVG_AVG_B)} [nT]\n")

    f.write("\nFlow Speed: \n")
    f.write(f"Mean: {np.mean(C_AVG_FLOW_SPEED)} [km/s]\n")
    f.write(f"Standard Deviation: {np.std(C_AVG_FLOW_SPEED)} [km/s]\n")
    f.write(f"Minimum: {np.min(C_AVG_FLOW_SPEED)} [km/s]\n")
    f.write(f"Maximum: {np.max(C_AVG_FLOW_SPEED)} [km/s]\n")

    f.write("\nProton Density: \n")
    f.write(f"Mean: {np.mean(C_AVG_PROTON_DENSITY)} [n/cc]\n")
    f.write(f"Standard Deviation: {np.std(C_AVG_PROTON_DENSITY)} [n/cc]\n")
    f.write(f"Minimum: {np.min(C_AVG_PROTON_DENSITY)} [n/cc]\n")
    f.write(f"Maximum: {np.max(C_AVG_PROTON_DENSITY)} [n/cc]\n")

    f.write("\nSYM_H: \n")
    f.write(f"Mean: {np.mean(C_AVG_SYM_H)} [nT]\n")
    f.write(f"Standard Deviation: {np.std(C_AVG_SYM_H)} [nT]\n")
    f.write(f"Minimum: {np.min(C_AVG_SYM_H)} [nT]\n")
    f.write(f"Maximum: {np.max(C_AVG_SYM_H)} [nT]\n")

In [12]:
plt.title("RBSP - Closest POES L Shell Comparison")
plt.xlabel("RBSP L-Shell")
plt.ylabel("Closest POES L-Shell")
plt.hlines(y=4, xmin=1, xmax=7, color="black")
plt.vlines(x=4, ymin=1, ymax=7, color="black")

plt.scatter(C_RBSP_LSTAR, C_POES_LSTAR)

print(f"Mean difference: {np.mean(C_POES_LSTAR - C_RBSP_LSTAR)}")
print(f"Standard deviation of difference {np.std(C_POES_LSTAR - C_RBSP_LSTAR)}")
print(f"Maximum difference : {np.max(C_POES_LSTAR - C_RBSP_LSTAR)}")

Mean difference: -0.000989374702853715
Standard deviation of difference 0.058858771499251804
Maximum difference : 0.09999987558131096


In [21]:
# Stage 3 Continued, Removing solar proton events!

VERSION = "v1c"
FIELD_MODEL = "T89"

pdata_folder = os.path.abspath(r"./../processed_data/chorus_neural_network/")

STAGE_2_folder = os.path.join(pdata_folder, "STAGE_2", VERSION)
STAGE_3_folder = os.path.join(pdata_folder, "STAGE_3", VERSION)


CONJUNCTIONS_REFS = np.load(
    os.path.join(STAGE_2_folder, rf"CONJUNCTIONS_{VERSION}_{FIELD_MODEL}.npz")
)

CONJUNCTIONS = CONJUNCTIONS_REFS["CONJUNCTIONS"]

CONJUNCTIONS_REFS.close()

SOLAR_PROTON_EVENT_LIST = pd.read_csv(
    os.path.join(pdata_folder, r"SOLAR_PROTON_EVENT_LIST_1976_2024.csv")
)

In [22]:
"""CONJUNCTION =  [UNIX_TIME,
L,
MLT,
*FLUX_SPECTRUM,
candidate[0], #TIME
candidate[1], #L
candidate[2], #MLT
candidate[3], #del_MLT
candidate[4], #CHORUS
AVG_SME,
AVG_AVG_B,
AVG_FLOW_SPEED,
AVG_PROTON_DENSITY,
AVG_SYM_H]"""

order_to_sort_conjunctions = np.argsort(
    CONJUNCTIONS[:, 0]
)  # Sorted based on POES Conjunction time!
SORTED_CONJUNCTIONS = CONJUNCTIONS[order_to_sort_conjunctions, :]

print(f"Starting shape of conjunctions list: {SORTED_CONJUNCTIONS.shape}")

SORTED_POES_CONJUNCTION_TIMES = SORTED_CONJUNCTIONS[:, 0]

START_OF_SEP_EVENTS_UTC = SOLAR_PROTON_EVENT_LIST["START"]
END_OF_SEP_EVENTS_UTC = SOLAR_PROTON_EVENT_LIST["END"]
ZIPPED_EVENTS = list(zip(START_OF_SEP_EVENTS_UTC, END_OF_SEP_EVENTS_UTC))

print("Removing high energy solar proton events!")

for SEP_EVENT in tqdm.tqdm(range(len(ZIPPED_EVENTS))):

    START = ZIPPED_EVENTS[SEP_EVENT][0].strip()
    END = ZIPPED_EVENTS[SEP_EVENT][1].strip()

    START_YMDHMS = {
        "year": int(START[0:4]),
        "month": int(START[5:7]),
        "day": int(START[8:10]),
        "hour": int(START[11:13]),
        "minute": int(START[13:15]),
        "second": 0,
    }
    END_YMDHMS = {
        "year": int(END[0:4]),
        "month": int(END[5:7]),
        "day": int(END[8:10]),
        "hour": int(END[11:13]),
        "minute": int(END[13:15]),
        "second": 0,
    }

    START_UNIX = astropy.time.Time(START_YMDHMS, format="ymdhms", scale="utc").unix
    END_UNIX = astropy.time.Time(END_YMDHMS, format="ymdhms", scale="utc").unix

    RANGE_TO_REMOVE = np.searchsorted(
        a=SORTED_POES_CONJUNCTION_TIMES, v=[START_UNIX, END_UNIX]
    )

    SORTED_CONJUNCTIONS = np.vstack(
        (
            SORTED_CONJUNCTIONS[0 : RANGE_TO_REMOVE[0], :],
            SORTED_CONJUNCTIONS[RANGE_TO_REMOVE[1] :, :],
        )
    )

print("Finished removing high energy solar proton events!")

print("Saving!")

CLEANED_CONJUNCTIONS = SORTED_CONJUNCTIONS  # Should be cleaned by now!

np.savez(
    file=os.path.join(STAGE_3_folder, rf"CLEANED_CONJUNCTIONS_{VERSION}_{FIELD_MODEL}.npz"),
    CONJUNCTIONS=CLEANED_CONJUNCTIONS,
)

C_POES_TIME = CLEANED_CONJUNCTIONS[:, 0]
C_POES_LSTAR = CLEANED_CONJUNCTIONS[:, 1]
C_POES_MLT = CLEANED_CONJUNCTIONS[:, 2]
C_POES_FLUX = CLEANED_CONJUNCTIONS[:, 3:-10]
C_RBSP_TIME = CLEANED_CONJUNCTIONS[:, -10]
C_RBSP_LSTAR = CLEANED_CONJUNCTIONS[:, -9]
C_RBSP_DEL_MLT = CLEANED_CONJUNCTIONS[:, -8]
C_RBSP_UPPER_BAND = CLEANED_CONJUNCTIONS[:, -7]
C_RBSP_LOWER_BAND = CLEANED_CONJUNCTIONS[:, -6]
C_AVG_SME = CLEANED_CONJUNCTIONS[:, -5]
C_AVG_AVG_B = CLEANED_CONJUNCTIONS[:, -4]
C_AVG_FLOW_SPEED = CLEANED_CONJUNCTIONS[:, -3]
C_AVG_PROTON_DENSITY = CLEANED_CONJUNCTIONS[:, -2]
C_AVG_SYM_H = CLEANED_CONJUNCTIONS[:, -1]

print("Creating documentation of dataset!")


with open(os.path.join(STAGE_3_folder, rf"CLEANED_CONJUNCTIONS_{VERSION}_{FIELD_MODEL}.txt"), "w",) as f:

    f.write("\nConjunctions:\n")
    f.write(f"Number of conjunctions: {CLEANED_CONJUNCTIONS.shape[0]} [#]\n")
    f.write(
        f"Number lost from cleaning solar proton events: {CONJUNCTIONS.shape[0] - CLEANED_CONJUNCTIONS.shape[0]} [#]\n"
    )
    f.write(f"Minimum RBSP Time: {np.min(C_RBSP_TIME)} [seconds since unix epoch]\n")
    f.write(f"Maximum RBSP Time: {np.max(C_RBSP_TIME)} [seconds since unix epoch]\n")
    f.write(f"Minimum POES Time: {np.min(C_POES_TIME)} [seconds since unix epoch]\n")
    f.write(f"Maximum POES Time: {np.max(C_POES_TIME)} [seconds since unix epoch]\n")

    f.write("\nL:\n")
    f.write(f"Mean Difference: {np.mean(C_POES_LSTAR - C_RBSP_LSTAR)} [L]\n")
    f.write(
        f"Standard deviation of Difference {np.std(C_POES_LSTAR - C_RBSP_LSTAR)} [L]\n"
    )
    f.write(
        f"Minimum Absolute Difference : {np.min(np.abs(C_POES_LSTAR - C_RBSP_LSTAR))} [L]\n"
    )
    f.write(
        f"Maximum Absolute Difference : {np.max(np.abs(C_POES_LSTAR - C_RBSP_LSTAR))} [L]\n"
    )

    f.write("\nMLT: \n")
    f.write(f"Mean Absolute Difference: {np.mean(C_RBSP_DEL_MLT)} [MLT]\n")
    f.write(
        f"Standard deviation of Absolute Difference {np.std(C_RBSP_DEL_MLT)} [MLT]\n"
    )
    f.write(f"Minimum Absolute Difference : {np.min(np.abs(C_RBSP_DEL_MLT))} [MLT]\n")
    f.write(f"Maximum Absolute Difference : {np.max(np.abs(C_RBSP_DEL_MLT))} [MLT]\n")

    f.write("\nTime: \n")
    f.write(f"Mean Difference: {np.mean(C_POES_TIME - C_RBSP_TIME)} [s]\n")
    f.write(
        f"Standard deviation of Difference {np.std(C_POES_TIME - C_RBSP_TIME)} [s]\n"
    )
    f.write(
        f"Minimum Absolute Difference : {np.min(np.abs(C_POES_TIME - C_RBSP_TIME))} [s]\n"
    )
    f.write(
        f"Maximum Absolute Difference : {np.max(np.abs(C_POES_TIME - C_RBSP_TIME))} [s]\n"
    )

    f.write("\nUpper Band Chorus: \n")
    f.write(f"Mean: {np.mean(C_RBSP_UPPER_BAND)} [pT]\n")
    f.write(f"Standard Deviation: {np.std(C_RBSP_UPPER_BAND)} [pT]\n")
    f.write(f"Minimum: {np.min(C_RBSP_UPPER_BAND)} [pT]\n")
    f.write(f"Maximum: {np.max(C_RBSP_UPPER_BAND)} [pT]\n")

    f.write("\nLower Band Chorus: \n")
    f.write(f"Mean: {np.mean(C_RBSP_LOWER_BAND)} [pT]\n")
    f.write(f"Standard Deviation: {np.std(C_RBSP_LOWER_BAND)} [pT]\n")
    f.write(f"Minimum: {np.min(C_RBSP_LOWER_BAND)} [pT]\n")
    f.write(f"Maximum: {np.max(C_RBSP_LOWER_BAND)} [pT]\n")

    f.write("\nSME: \n")
    f.write(f"Mean: {np.mean(C_AVG_SME)} [nT]\n")
    f.write(f"Standard Deviation: {np.std(C_AVG_SME)} [nT]\n")
    f.write(f"Minimum: {np.min(C_AVG_SME)} [nT]\n")
    f.write(f"Maximum: {np.max(C_AVG_SME)} [nT]\n")

    f.write("\nAVG_B: \n")
    f.write(f"Mean: {np.mean(C_AVG_AVG_B)} [nT]\n")
    f.write(f"Standard Deviation: {np.std(C_AVG_AVG_B)} [nT]\n")
    f.write(f"Minimum: {np.min(C_AVG_AVG_B)} [nT]\n")
    f.write(f"Maximum: {np.max(C_AVG_AVG_B)} [nT]\n")

    f.write("\nFlow Speed: \n")
    f.write(f"Mean: {np.mean(C_AVG_FLOW_SPEED)} [km/s]\n")
    f.write(f"Standard Deviation: {np.std(C_AVG_FLOW_SPEED)} [km/s]\n")
    f.write(f"Minimum: {np.min(C_AVG_FLOW_SPEED)} [km/s]\n")
    f.write(f"Maximum: {np.max(C_AVG_FLOW_SPEED)} [km/s]\n")

    f.write("\nProton Density: \n")
    f.write(f"Mean: {np.mean(C_AVG_PROTON_DENSITY)} [n/cc]\n")
    f.write(f"Standard Deviation: {np.std(C_AVG_PROTON_DENSITY)} [n/cc]\n")
    f.write(f"Minimum: {np.min(C_AVG_PROTON_DENSITY)} [n/cc]\n")
    f.write(f"Maximum: {np.max(C_AVG_PROTON_DENSITY)} [n/cc]\n")

    f.write("\nSYM_H: \n")
    f.write(f"Mean: {np.mean(C_AVG_SYM_H)} [nT]\n")
    f.write(f"Standard Deviation: {np.std(C_AVG_SYM_H)} [nT]\n")
    f.write(f"Minimum: {np.min(C_AVG_SYM_H)} [nT]\n")
    f.write(f"Maximum: {np.max(C_AVG_SYM_H)} [nT]\n")

print("Finished!")
print(f"Ending shape of conjunctions : {CLEANED_CONJUNCTIONS.shape}")

Starting shape of conjunctions list: (969370, 21)
Removing high energy solar proton events!


100%|████████████████████████████████████████████████████████████████████████████████| 309/309 [00:13<00:00, 23.31it/s]


Finished removing high energy solar proton events!
Saving!
Creating documentation of dataset!
Finished!
Ending shape of conjunctions : (962732, 21)


In [ ]:
# Stage 4, Create datasets used for training, testing, etc

VERSION = "v1c"
FIELD_MODEL = "T89"
MODEL_TYPE = "UPPER_BAND"

STAGE_3_folder = os.path.join(pdata_folder, "STAGE_3", VERSION)

CONJUNCTIONS_REFS = np.load(
    file=os.path.join(STAGE_3_folder, rf"CLEANED_CONJUNCTIONS_{VERSION}_{FIELD_MODEL}.npz")
)

CONJUNCTIONS = CONJUNCTIONS_REFS["CONJUNCTIONS"]

CONJUNCTIONS_REFS.close()

In [ ]:
print(CONJUNCTIONS.shape)

C_RBSP_TIME = CONJUNCTIONS[:, -10]

jan1_unix = astropy.time.Time(
    {"year": 2016, "month": 1, "day": 1, "hour": 0, "minute": 0, "second": 0},
    format="ymdhms",
    scale="utc",
).unix
apr1_unix = astropy.time.Time(
    {"year": 2016, "month": 4, "day": 1, "hour": 0, "minute": 0, "second": 0},
    format="ymdhms",
    scale="utc",
).unix

where_between_feb1_apr1_2013 = (jan1_unix < C_RBSP_TIME) & (C_RBSP_TIME < apr1_unix)

train_test_subset_selected = ~where_between_feb1_apr1_2013
validation_subset_selected = where_between_feb1_apr1_2013

print(
    f"Number of conjunctions between feb1 and apr1 2013: {np.count_nonzero(where_between_feb1_apr1_2013)}"
)

C_POES_TIME = np.expand_dims(CONJUNCTIONS[train_test_subset_selected, 0], axis=1)
C_POES_LSTAR = np.expand_dims(CONJUNCTIONS[train_test_subset_selected, 1], axis=1)
C_POES_MLT = np.expand_dims(CONJUNCTIONS[train_test_subset_selected, 2], axis=1)
C_POES_FLUX = CONJUNCTIONS[train_test_subset_selected, 3:-10]
C_RBSP_TIME = np.expand_dims(CONJUNCTIONS[train_test_subset_selected, -10], axis=1)
C_RBSP_LSTAR = np.expand_dims(CONJUNCTIONS[train_test_subset_selected, -9], axis=1)
C_RBSP_DEL_MLT = np.expand_dims(CONJUNCTIONS[train_test_subset_selected, -8], axis=1)
C_RBSP_UPPER_BAND = np.expand_dims(CONJUNCTIONS[train_test_subset_selected, -7], axis=1)
C_RBSP_LOWER_BAND = np.expand_dims(CONJUNCTIONS[train_test_subset_selected, -6], axis=1)
C_AVG_SME = np.expand_dims(CONJUNCTIONS[train_test_subset_selected, -5], axis=1)
C_AVG_AVG_B = np.expand_dims(CONJUNCTIONS[train_test_subset_selected, -4], axis=1)
C_AVG_FLOW_SPEED = np.expand_dims(CONJUNCTIONS[train_test_subset_selected, -3], axis=1)
C_AVG_PROTON_DENSITY = np.expand_dims(
    CONJUNCTIONS[train_test_subset_selected, -2], axis=1
)
C_AVG_SYM_H = np.expand_dims(CONJUNCTIONS[train_test_subset_selected, -1], axis=1)

print(C_RBSP_TIME.shape)
print(C_RBSP_LSTAR.shape)
print(C_RBSP_UPPER_BAND.shape)
print(C_RBSP_LOWER_BAND.shape)
print(C_POES_TIME.shape)
print(C_POES_LSTAR.shape)
print(C_POES_MLT.shape)
print(C_RBSP_DEL_MLT.shape)
print(C_POES_FLUX.shape)
print(C_AVG_SME.shape)
print(C_AVG_AVG_B.shape)
print(C_AVG_FLOW_SPEED.shape)
print(C_AVG_PROTON_DENSITY.shape)
print(C_AVG_SYM_H.shape)

mean_LSTAR = np.nanmean(C_POES_LSTAR)
std_LSTAR = np.std(C_POES_LSTAR)

mean_fluxes = np.expand_dims(np.nanmean(C_POES_FLUX, axis=0), axis=0)
std_fluxes = np.expand_dims(np.nanstd(C_POES_FLUX, axis=0), axis=0)

mean_sme = np.nanmean(C_AVG_SME)
std_sme = np.std(C_AVG_SME)

mean_avg_b = np.nanmean(C_AVG_AVG_B)
std_avg_b = np.std(C_AVG_AVG_B)

mean_flow_speed = np.nanmean(C_AVG_FLOW_SPEED)
std_flow_speed = np.std(C_AVG_FLOW_SPEED)

mean_avg_proton_density = np.nanmean(C_AVG_PROTON_DENSITY)
std_avg_proton_density = np.std(C_AVG_PROTON_DENSITY)

mean_avg_sym_h = np.nanmean(C_AVG_SYM_H)
std_avg_sym_h = np.std(C_AVG_SYM_H)

FEATURES = np.hstack(
    (
        (C_POES_LSTAR - mean_LSTAR) / std_LSTAR,
        np.sin((C_POES_MLT * 2 * np.pi) / 24.0),
        np.cos((C_POES_MLT * 2 * np.pi) / 24.0),
        ((C_POES_FLUX - mean_fluxes) / std_fluxes),
        (C_AVG_SME - mean_sme) / std_sme,
        (C_AVG_AVG_B - mean_avg_b) / std_avg_b,
        # (C_AVG_FLOW_SPEED - mean_flow_speed) / std_flow_speed,
        # (C_AVG_PROTON_DENSITY - mean_avg_proton_density) / std_avg_proton_density,
        (C_AVG_SYM_H - mean_avg_sym_h) / std_avg_sym_h,
    )
)


# SMALL VALIDATION SET:
C_POES_TIME_V = np.expand_dims(CONJUNCTIONS[validation_subset_selected, 0], axis=1)
C_POES_L_V = np.expand_dims(CONJUNCTIONS[validation_subset_selected, 1], axis=1)
C_POES_MLT_V = np.expand_dims(CONJUNCTIONS[validation_subset_selected, 2], axis=1)
C_POES_FLUX_V = CONJUNCTIONS[validation_subset_selected, 3:-10]
C_RBSP_TIME_V = np.expand_dims(CONJUNCTIONS[validation_subset_selected, -10], axis=1)
C_RBSP_L_V = np.expand_dims(CONJUNCTIONS[validation_subset_selected, -9], axis=1)
C_RBSP_DEL_MLT_V = np.expand_dims(CONJUNCTIONS[validation_subset_selected, -8], axis=1)
C_RBSP_UPPER_BAND_V = np.expand_dims(
    CONJUNCTIONS[validation_subset_selected, -7], axis=1
)
C_RBSP_LOWER_BAND_V = np.expand_dims(
    CONJUNCTIONS[validation_subset_selected, -6], axis=1
)
C_AVG_SME_V = np.expand_dims(CONJUNCTIONS[validation_subset_selected, -5], axis=1)
C_AVG_AVG_B_V = np.expand_dims(CONJUNCTIONS[validation_subset_selected, -4], axis=1)
C_AVG_FLOW_SPEED_V = np.expand_dims(
    CONJUNCTIONS[validation_subset_selected, -3], axis=1
)
C_AVG_PROTON_DENSITY_V = np.expand_dims(
    CONJUNCTIONS[validation_subset_selected, -2], axis=1
)
C_AVG_SYM_H_V = np.expand_dims(CONJUNCTIONS[validation_subset_selected, -1], axis=1)

print(C_RBSP_TIME_V.shape)
print(C_RBSP_L_V.shape)
print(C_RBSP_UPPER_BAND_V.shape)
print(C_RBSP_LOWER_BAND_V.shape)
print(C_POES_TIME_V.shape)
print(C_POES_L_V.shape)
print(C_POES_MLT_V.shape)
print(C_RBSP_DEL_MLT_V.shape)
print(C_POES_FLUX_V.shape)
print(C_AVG_SME_V.shape)
print(C_AVG_AVG_B_V.shape)
print(C_AVG_FLOW_SPEED_V.shape)
print(C_AVG_PROTON_DENSITY_V.shape)
print(C_AVG_SYM_H_V.shape)


VALIDATION_FEATURES = np.hstack(
    (
        (C_POES_L_V - mean_LSTAR) / std_LSTAR,
        np.sin((C_POES_MLT_V * 2 * np.pi) / 24.0),
        np.cos((C_POES_MLT_V * 2 * np.pi) / 24.0),
        ((C_POES_FLUX_V - mean_fluxes) / std_fluxes),
        (C_AVG_SME_V - mean_sme) / std_sme,
        (C_AVG_AVG_B_V - mean_avg_b) / std_avg_b,
        # (C_AVG_FLOW_SPEED_V - mean_flow_speed) / std_flow_speed,
        # (C_AVG_PROTON_DENSITY_V - mean_avg_proton_density) / std_avg_proton_density,
        (C_AVG_SYM_H_V - mean_avg_sym_h) / std_avg_sym_h,
    )
)

if MODEL_TYPE == "UPPER_BAND":
    MODEL_LABELS = C_RBSP_UPPER_BAND
    MODEL_LABELS_V = C_RBSP_UPPER_BAND_V

elif MODEL_TYPE == "LOWER_BAND":
    MODEL_LABELS = C_RBSP_LOWER_BAND
    MODEL_LABELS_V = C_RBSP_LOWER_BAND_V

np.savez(
    f"./../processed_data/chorus_neural_network/STAGE_4/{VERSION}/MODEL_READY_DATA_{VERSION}_{FIELD_MODEL}_{MODEL_TYPE}_2.npz",
    FEATURES=FEATURES,
    LABELS=MODEL_LABELS,
    VALIDATION_FEATURES=VALIDATION_FEATURES,
    VALIDATION_LABELS=MODEL_LABELS_V,
    TRAINING_MLT=C_POES_MLT,
    MEAN_FLUXES=mean_fluxes,
    STD_FLUXES=std_fluxes,
    MEAN_SME=mean_sme,
    STD_SME=std_sme,
    MEAN_AVG_B=mean_avg_b,
    STD_AVG_B=std_avg_b,
    MEAN_FLOW_SPEED=mean_flow_speed,
    STD_FLOW_SPEED=std_flow_speed,
    MEAN_AVG_PROTON_DENSITY=mean_avg_proton_density,
    STD_AVG_PROTON_DENSITY=std_avg_proton_density,
    MEAN_AVG_SYM_H=mean_avg_sym_h,
    STD_AVG_SYM_H=std_avg_sym_h,
)